In [94]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro as sh
from scipy.stats import wilcoxon as wil

# Ejercicio XII.1

### Se desea contrastar la hipótesis de normalidad de una población de la cual se ha extraído la siguiente muestra: 12,3; 11,0; 10,7; 12,4; 11,7; 13,1; 9,9; 12,6; 11,8; 10,2 y 10,5.

Se toma un nivel de significación, $\alpha$, del 5 %:

In [95]:
alfa = 0.05

En primer lugar, se define una lista con la muestra:

In [96]:
muestra = [12.3, 11.0, 10.7, 12.4, 11.7, 13.1, 9.9, 12.6, 11.8, 10.2, 10.5]

Y se convierte en una matriz de NumPy:

In [97]:
muestra = np.array(muestra)
muestra

array([12.3, 11. , 10.7, 12.4, 11.7, 13.1,  9.9, 12.6, 11.8, 10.2, 10.5])

Para aplicar el contraste de Shapiro-Wilk se necesita, en primer lugar, determinar el tamaño muestral:

In [98]:
n = muestra.size
n

11

Para $n = 11$, los valores tabulados de Shapiro-Wilk -obtenidos a partir de la tabla correspondiente- resultan ser los siguientes:

In [99]:
coeficientes = [0.5601, 0.3315, 0.2260, 0.1429, 0.0695]
coeficientes = np.array(coeficientes)

Se calcula asimismo la mediana de la muestra:

In [100]:
mediana = np.median(muestra)
mediana

11.7

Se parte la matriz muestral en dos submatrices, una que contenga los valores superiores a la mediana y otra con
los inferiores, y se ordenan de manera inversa para posteriormente poder calcular las diferencias entre los
pares de valores que equidistan entre sí:

In [101]:
submuestra1 = np.sort(muestra[muestra > mediana])
submuestra1 = submuestra1[::-1] #Ordena los elementos decrecientemente

submuestra2 = np.sort(muestra[muestra < mediana])

diferencias = submuestra1 - submuestra2

Las diferencias se multiplican por los coeficientes para obtener los valores que, sumados, proporcionan el
numerador del estadístico experimental de Shapiro-Wilk:

In [102]:
sumandos = diferencias * coeficientes

Todas estas variables se pueden presentar de forma resumida en una tabla, construida como marco de datos de Pandas, para facilitar su visualización:

In [103]:
tablaResumen = pd.DataFrame({'x_n-i+1': submuestra1,
                             'x_i': submuestra2,
                             'x_n-i+1 - x_i': diferencias, 
                             'a_n-i+i': coeficientes,
                             '(x_n-i+1 - x_i)(a_n-i+1)': sumandos
                             })

tablaResumen

,x_n-i+1,x_i,x_n-i+1 - x_i,a_n-i+i,(x_n-i+1 - x_i)(a_n-i+1)
0,13.1,9.9,3.2,0.5601,1.79232
1,12.6,10.2,2.4,0.3315,0.79560
2,12.4,10.5,1.9,0.2260,0.42940
3,12.3,10.7,1.6,0.1429,0.22864
4,11.8,11.0,0.8,0.0695,0.05560


Se calcula el valor del estadístico experimental, $W_{exp}$:

In [104]:
Wexp = round(sum(sumandos)**2/((n-1)*np.var(muestra, ddof = 1)), 3) #Con un grado de libertad calcula la cuasivarianza
Wexp

0.949

Si se consulta en la tabla correspondiente, el valor crítico para un nivel de significación del 5 %, $W_{11; 0,05}$, resulta ser:

In [105]:
Wc = 0.85

Dado que $W_{exp} > W_{n, \alpha}$, no se cae en la región crítica y se puede asumir la hipótesis de que la muestra tiene una estructura normal:

In [106]:
Wexp > Wc

True

Por otro lado, Python dispone de una manera automática de realizar el contraste de Shapiro-Wilk:

In [107]:
Wexp, p = sh(muestra)

print(f'Wexp = {Wexp:.3f}, p-valor = {p:.3f}')

if p > alfa: 
  print('No se puede rechazar la hipótesis nula, por lo que es probable que la población siga una distribución normal.')
else:
  print('La población no sigue una distribución normal.')

Wexp = 0.950, p-valor = 0.640
No se puede rechazar la hipótesis nula, por lo que es probable que la población siga una distribución normal.


# Ejercicio XII.2

### Se desea contrastar si la mediana de la población de la cual se ha extraído la siguiente MAS vale 5: 4, 5, 6, 5, 3, 4, 2, 7, 6, 5, 4, 3, 8, 8, 9, 4, 6, 7, 2, 5, 6.

Se plantea el siguiente contraste T de Wilcoxon:

$$
\left\{
  \begin{array}{ll} 
    H_{0}: Me = 5 \\
    H_{1}: Me \ne 5
  \end{array}
\right.
$$

Se supone un nivel de significación, $\alpha$, del 5 %:

In [108]:
alfa = 0.05
me0 = 5

Como es habitual, se define una lista con la muestra y se convierte en una matriz de NumPy:

In [109]:
muestra = [4, 5, 6, 5, 3, 4, 2, 7, 6, 5, 4, 3, 8, 8, 9, 4, 6, 7, 2, 5, 6]
muestra = np.array(muestra)
muestra

array([4, 5, 6, 5, 3, 4, 2, 7, 6, 5, 4, 3, 8, 8, 9, 4, 6, 7, 2, 5, 6])

Si se comprueba el valor de la mediana muestral:

In [110]:
mediana = np.median(muestra)
mediana

5.0

Se observa que coincide con el valor utilizado como hipótesis nula, $Me_{0} = 5$.

En primer lugar, se calculan las diferencias entre cada elemento de la muestra y la mediana hipotética, $x_{i}-Me_{0}$:

In [111]:
diferencias = muestra - me0
diferencias

array([-1,  0,  1,  0, -2, -1, -3,  2,  1,  0, -1, -2,  3,  3,  4, -1,  1,
        2, -3,  0,  1])

A continuación, se eliminan las diferencias nulas, se obtiene el valor absoluto de las restantes y se las ordena en orden ascendente:

In [112]:
absolutas = np.sort(abs(diferencias[diferencias != 0]))
absolutas

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4])

Se determinan los valores únicos para, a partir de ellos, estimar cuales son los rangos promedio que les corresponden:

In [113]:
unicos = np.unique(absolutas)
unicos

array([1, 2, 3, 4])

In [114]:
"""
Calcula la media de los índices -en Python empiezan en 0, de ahí el '+ 1'- que ocupan los valores de 'absolutas' para
cada valor en 'unicos' -media de los índices de los unos, de 1 a 8; de los doses, de 9 a 12; etc.-
"""

rangos = np.array([np.mean(np.array(np.where(absolutas == valor)) + 1) for valor in unicos])
rangos

array([ 4.5, 10.5, 14.5, 17. ])

Se define la siguiente función, que permite calcular los sumatorios de los rangos promedio en función del signo del valor de las diferencias:

In [115]:
def estadistico(diferencias, rangos): #Matriz con diferencias positivas o negativas, nunca ambas
    
    lista = []
    
    for diferencia in diferencias:
        for rango in rangos:
            if abs(diferencia) == np.array(np.where(rangos == rango)) + 1: lista.append(rango)
                
    """
    Cada diferencia -en valor absoluto- sirve de índice para el rango: si es un 1, se añade un 4,5 a la lista; 
    si es un 2, un 10,5, etc.
    """
    
    sumatorio = np.sum(np.array(lista))
    
    return sumatorio

Si se parte la matriz muestral en dos submatrices, una con los valores negativos y otra con los positivos, se pueden obtener los dos posibles valores, $T^{-}$ y $T^{+}$, del estadístico experimental, $T_{exp}$:

In [116]:
diferenciasNeg = diferencias[diferencias < 0]

Tneg = estadistico(diferenciasNeg, rangos)
Tneg

68.0

In [117]:
diferenciasPos = diferencias[diferencias > 0]

Tpos = estadistico(diferenciasPos, rangos)
Tpos

85.0

En principio, el valor de $T_{exp}$ sería el mínimo entre $T^{-}$ y $T^{+}$:

In [118]:
Texp = min(Tneg, Tpos)
Texp

68.0

Finalmente, se calcula el número de diferencias no nulas, $n'$:

In [119]:
n = absolutas.size
n

17

Si se consulta en la tabla correspondiente, el valor crítico para un nivel de significación del 5 %, $T_{17; 0,05}$, resulta ser:

In [120]:
Tc = 41

Dado que $T_{exp} > T_{c}$, no se cae en la región crítica y por tanto no se puede rechazar $H_{0}$:

In [121]:
Texp > Tc

True

Python dispone de una manera automatizada de realizar el contraste T de Wilcoxon:

In [122]:
Texp, p = wil(muestra - me0, correction = False, mode = 'approx')

print(f'Texp = {Texp:.1f}, p-valor = {p:.3f}')

if p > alfa: 
  print('No se puede rechazar la hipótesis nula.')
else:
  print('Se acepta la hipótesis alternativa.')

Texp = 68.0, p-valor = 0.683
No se puede rechazar la hipótesis nula.
